In [ ]:
#get a small sample of the data to test the code from caucassian and african 

In [ ]:
'''
extract all identities that have more than #num_images/id images, 
then uniformally get num_identity identities from each race
then uniformally get num_images/id images from each identity
then split into train and test
then resize data to 112*112
'''

In [19]:
import random
def get_data_uniformly(data, num_image=10):
    random_choce = random.sample(data, num_image)
    return random_choce

In [27]:
import pandas as pd
import os

'''
data frame: id_name, image_name, race 
data frame: id_name, id_name_count, image_name, race
race: caucassian = 0, african = 1
'''
race_dict = {'caucasian': 0, 'african': 1}


def get_dataset(race_folder, race, image_per_id= 10, image_per_id_limit=10):
    coloumns = ['image_name', 'id', 'race']
    balanced_dataset = []
    race_idx = race_dict[race]

    for id_folder in os.listdir(race_folder):
        all_id_image = []

        if not 'm.' in id_folder:
            continue
        id_path = os.path.join(race_folder, id_folder)
        id = id_folder.split("/")[0]

        num_image = 0

        for image in os.listdir(id_path):
            num_image += 1
        if num_image < image_per_id_limit:
            continue
        else:
            for image in os.listdir(id_path):
                image_path = os.path.join(id_path, image)
                all_id_image.append(image)
        uniform_subset = get_data_uniformly(all_id_image, image_per_id)

        for i, image in enumerate(uniform_subset):
            image_name = image
            balanced_dataset.append([image_name, id, race_idx])
    balanced_dataset = pd.DataFrame(balanced_dataset, columns=coloumns)
    return balanced_dataset
        

In [28]:
caucassian_balanced_data = get_dataset('../ore_code/race_per_7000/Caucasian', 'caucasian')

In [30]:
#group by id and show how many id
caucassian_balanced_data.groupby('id').count()

,image_name,race
id,,
m.011_0k,10,10
m.011_pk,10,10
m.011m1y,10,10
m.011s_x,10,10
m.0120mb,10,10
...,...,...
m.0sw7b,10,10
m.0t5w1q8,10,10
m.0tpbzwn,10,10


In [36]:
'''
pick identity uniformly from pd
'''
def pick_identity_uniformly(data, num_id=3000):
    id_list = data['id'].unique()
    random_id_list = random.sample(list(id_list), num_id)
    #get all data from random_id_list
    random_choice = data[data['id'].isin(random_id_list)]
    random_choice = random_choice.reset_index(drop=True)
    return random_choice

In [37]:
caucassian_identity_uniformly = pick_identity_uniformly(caucassian_balanced_data)

In [39]:
print(len(caucassian_identity_uniformly))

30000


In [42]:
def save_id_selection_per_race(identity_pd, save_path, file_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    identity_pd.to_csv(save_path + file_path, index=False)

In [44]:
save_id_selection_per_race(caucassian_identity_uniformly, save_path='./data', file_path='/caucassian_id_selection.csv')

In [52]:
import os
import shutil

def get_image_to_folder(identity_selection, data_path, save_path):
    #save to ./identity/image_name.png
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for _, row in identity_selection.iterrows():
        image_name = row['image_name']
        image_name.replace('.jpy', '.png')
        identity = row['id']
        # if identity == 'm.034qpf':
        #     continue
        source_path = os.path.join(data_path, identity, image_name)
        identity_save_path = os.path.join(save_path, identity)

        # Create a subdirectory for the identity if it doesn't exist
        if not os.path.exists(identity_save_path):
            os.makedirs(identity_save_path)

        destination_path = os.path.join(identity_save_path, image_name)

        #check if image already exist
        if os.path.exists(destination_path):
            continue
        # Copy the image from the source to the destination
        shutil.copyfile(source_path, destination_path)

    print('Finished copying images to folder')


In [54]:
!pwd

/Users/nina/Desktop/SupContrast-ORE


In [55]:
get_image_to_folder(caucassian_identity_uniformly, 
                    data_path='/Users/nina/Desktop/ore_code/race_per_7000/Caucasian', 
                    save_path='./data/caucassian_id_selection/')

KeyboardInterrupt: 